# LightRay Analysys

In this notebook we will study light reflaction ans refraction, using LightRay analysis and the Snell and Fresnel equations.

In [ ]:
!pip install numpy
!pip install ipympl
!pip install matplotlib

In [ ]:
from math import *
import numpy as np
import matplotlib.pyplot as plt

if 'google.colab' in str(get_ipython()):
  %matplotlib inline
else:
  %matplotlib widget

## Create a the a LightRay graph class

In [ ]:
class LightRay:
    """Defines a LightRay vecto"""
    def __init__(self, r_type:int, angle_deg:int): 
        """ Intialize a LightRay vector

            Args:
                type: int
                    Ray r_type -> 0 = Inciddent, 1 = Reflected and 2 = Refracted
                angle: float
                    Angle from the normal in degrees
            
            Returns: LightRay object
        """
        self.type = r_type
        self.angle = angle_deg
        self.polarization = 'p'


class Material:
    """Material with properties related to light interaction"""
    
    def __init__(self, name:str,  refractive_index:str, thickness_mm:float):
        """ Intialize a Material

            Args:
                name: str
                refractive_index: float
                thickness_mm: float
            
            Returns: Material object
        """
        self.name = name
        self.refractive_index = refractive_index
        self.thickness = thickness_mm


class RayGraph:
    ''' LightRay graph for multiple interfaces'''

    # def __init__(self):
    #     self.figure, self.axis = plt.subplots()
    
    def __init__(self, hight: int, width: int):
        self.figure, self.axis = plt.subplots()
        self.axis.set_ylim(-hight//2, hight//2)
        self.axis.set_xlim(-width//2, width//2)
        self.axis.set_ylabel('hight (mm)')
        self.axis.set_xlabel('width (mm)')


        interface = self.axis.plot([-width//2, width//2], [0, 0], 
                                    color='gray', linestyle='dashed', alpha= 0.5, label= 'interface')
        normal = self.axis.plot([0, 0], [-width//2, width//2], 
                                color='yellow', linestyle='dashed', alpha= 0.5, label='normal')

        plt.legend()

        self.ray_color='blue'
        self.ray_alpha=0.1

    def plot_vector(self, vector: list[tuple[int,int],tuple[int,int]]):
        """ Plot a vector based on the start and end point
            
            Args: 
                vector: list[(start_x, start_y), (end_x, end_y)]
            """
        a = vector[0]
        b = vector[1]
        dx = b[0] - a[0]
        dy = b[1] - a[1]

        magnitude = sqrt(dx**2+dy**2)
        # head_length = magnitude * 0.05
        head_length = 0.2

        dx = dx / magnitude
        dy = dy / magnitude

        magnitude = magnitude - head_length

        self.axis.arrow(a[0], a[1], magnitude*dx, magnitude*dy, 
                        head_width=head_length, head_length=head_length, 
                        color=self.ray_color,alpha=self.ray_alpha)
    
    def plot_snell_refraction(self, incident_ray: LightRay, material_1: Material, material_2: Material):
        """Plot the refracted ray for a 2 material interface based on the snell law"""
        incident_x_start = -tan(radians(incident_ray.angle))*material_1.thickness
        incident_y_start = material_1.thickness
        incident_x_end = 0
        incident_y_end = 0

        self.axis.set_title('Snell analysis')
        self.plot_vector([(incident_x_start,incident_y_start),
                          (incident_x_end,incident_y_end)])

        resultant_ray = snell_analysis(incident_ray, material_1, material_2)


        resultant_x_start = 0
        resultant_y_start = 0
        resultant_x_end = tan(radians(resultant_ray.angle))*material_2.thickness

        if resultant_ray.type == 1: # Total internal reflection ray
            resultant_y_end = material_2.thickness
        elif resultant_ray.type == 2: # Refracted ray
            resultant_y_end = -material_2.thickness
    
        self.plot_vector([(resultant_x_start,resultant_y_start),
                        (resultant_x_end,resultant_y_end)])

    def show(self):
        plt.show()

## Snells Law

n2 * sin(teta_2) = n1 * sin(teta_1)

In [ ]:
def snell_analysis(incident_ray: LightRay, material_1: Material, material_2: Material) -> LightRay:
    """ Calculate the refracted ray using the snells law"""

    n1 = material_1.refractive_index # Refractive index of the first materia
    n2 = material_2.refractive_index # Refractive index of the second material

    argument = n1*sin(radians(incident_ray.angle))/n2

    if argument <=  1:
        refraction_ang = asin(argument)
        resultant_ray = LightRay(2, degrees(refraction_ang))
    else:
        resultant_ray = LightRay(1, incident_ray.angle)

    return resultant_ray

def fresnel_analysis(incident_ray: LightRay, material_1: Material, material_2: Material) -> LightRay:
    """ Calculate the refracted ray using the snells law"""
    pass

In [ ]:
plt.close('all')

material_1 = Material('vacuum', 1.0, 5.0)
material_2 = Material('water', 1.33, 5.0)

angles = np.linspace(0.001, 89.99, num=10)

rayligh_graph = RayGraph(15,15)
rayligh_graph.ray_color = 'red'

for angle in angles:
    if rayligh_graph.ray_alpha + 1/len(angles) < 1:
        rayligh_graph.ray_alpha += 1/len(angles)
    incident_ray = LightRay(0, angle)
    rayligh_graph.plot_snell_refraction(incident_ray,material_1,material_2)

rayligh_graph.show()
    
material_1 = Material('optic_fiber', 1.33, 5)
material_2 = Material('fbg', 1.0, 5.0)

angles = np.linspace(0.001, 89.99, num=10)

rayligh_graph = RayGraph(15,15)
rayligh_graph.ray_color = 'red'

for angle in angles:
    if rayligh_graph.ray_alpha + 1/len(angles) < 1:
        rayligh_graph.ray_alpha += 1/len(angles)
    incident_ray = LightRay(0, angle)
    rayligh_graph.plot_snell_refraction(incident_ray,material_1,material_2)

rayligh_graph.show()
